# The Autonomous Vehicle Co-pilot
Here is a chatbot, empowered by GPT-3.5, running to help the vehicle driver. 

He is set as a co-pilot, analyzing the environment based on other multi-modality input, which has been processed as text so he can reason and identify potential risk. 


## Setup
Notice: if you tried to run this program on your linux server, 
and you are in China,
make sure that you opened the vpn. 
Otherwise the api account may be forbiddened!

In [1]:
import os
import openai

def read_key_from_file(filepath):
    with open(filepath, 'r') as file:
        return file.read().strip()
personal_key = read_key_from_file('apikey.cn')
organ = read_key_from_file('organization.cn')

openai.organization = organ
openai.api_key  = personal_key

TEMPERATURE = 0

ModuleNotFoundError: No module named 'openai'

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo-0613"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature= TEMPERATURE, # this is the degree of randomness of the model's output
    )
    return response
    # return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo",temp = TEMPERATURE):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature= temp, # this is the degree of randomness of the model's output
    )
    print(str(response.choices[0].message["content"]))
    return response.choices[0].message["content"]

In [ ]:
blind_guide = """ We are currently driving on a city street filled with green plants, with bicycle lanes on both sides of the road. On the left bicycle lane, there are three cyclists moving slowly, while on the right side, there are two cyclists riding at a faster pace.
Next, there is a spacious pedestrian walkway, with five pedestrians walking on the left side and four pedestrians on the right side, two of whom are jogging. Each side of the street is lined with banyan trees, providing ample shade for pedestrians to cool off.
We are heading towards an intersection. The traffic signal at the intersection is displaying a green light, indicating that we can proceed. Each direction at the intersection has only one lane, clearly marked. In front of us, there is a blue sedan driving at a moderate speed."""
chat_history = [
    {'role':'system', 'content':'You are a co-pilot AI assistant designed to help drivers understand their surroundings.'},
    
    {'role':'user', 'content':'What is our current situation on the road?'},
    { 'role':'system','content': blind_guide}
]


another = [
    {'role':'assistant', 'content':'We are currently driving on a city street adorned with greenery, with bicycle lanes present on both sides of the road. On our left, three cyclists are moving at a slow pace, while on the right, two cyclists are moving more quickly.'},

    {'role':'user', 'content':'What about pedestrians and the environment around us?'},

    {'role':'assistant', 'content':'Indeed, there is a spacious pedestrian walkway. On the left side, five pedestrians are walking and on the right side, four pedestrians, two of whom are jogging. The street is lined with banyan trees on either side, providing a shaded canopy for pedestrians.'},

    {'role':'user', 'content':'What lies ahead of us?'},

    {'role':'assistant', 'content':'We are heading towards an intersection. The traffic signal at the intersection currently displays a green light, indicating that we can proceed. Each direction at the intersection has a single lane, clearly marked. A blue sedan is driving at a moderate speed ahead of us.'}
]


InvalidRequestError: [{'role': 'system', 'content': 'You are a co-pilot AI assistant designed to help drivers understand their surroundings.'}, {'role': 'user', 'content': 'What is our current situation on the road?'}, {'role': 'system', 'content': ' We are currently driving on a city street filled with green plants, with bicycle lanes on both sides of the road. On the left bicycle lane, there are three cyclists moving slowly, while on the right side, there are two cyclists riding at a faster pace.\nNext, there is a spacious pedestrian walkway, with five pedestrians walking on the left side and four pedestrians on the right side, two of whom are jogging. Each side of the street is lined with banyan trees, providing ample shade for pedestrians to cool off.\nWe are heading towards an intersection. The traffic signal at the intersection is displaying a green light, indicating that we can proceed. Each direction at the intersection has only one lane, clearly marked. In front of us, there i

# Data Integration
 Firstly, you'll need a way to convert the input from the vehicle's sensors (like LIDAR, RADAR, cameras, etc.) into a format that the chatbot can understand. This might involve using object detection algorithms to identify objects and their positions, then converting this data into a text description of the scene.


## buggy chatbot here
I need to find a good programming way of coding here

In [ ]:
def chat_with_assistant(user_input, context):
    # Append the user input to the context
    context.append({'role':'user', 'content':f"{user_input}"})
    
    # Get the assistant's response
    response = get_completion_from_messages(context)
    
    # Append the assistant's response to the context
    context.append({'role':'assistant', 'content':f"{response}"})
    
    return context


# Chat with the assistant
context = chat_with_assistant('Hello, assistant!', chat_history)
print(context)

# Continue the conversation
context = chat_with_assistant('Tell me a joke', context)
print(context)


Hello! How can I assist you today?
[{'role': 'system', 'content': 'You are a co-pilot AI assistant designed to help drivers understand their surroundings.'}, {'role': 'user', 'content': 'What is our current situation on the road?'}, {'role': 'system', 'content': ' We are currently driving on a city street filled with green plants, with bicycle lanes on both sides of the road. On the left bicycle lane, there are three cyclists moving slowly, while on the right side, there are two cyclists riding at a faster pace.\nNext, there is a spacious pedestrian walkway, with five pedestrians walking on the left side and four pedestrians on the right side, two of whom are jogging. Each side of the street is lined with banyan trees, providing ample shade for pedestrians to cool off.\nWe are heading towards an intersection. The traffic signal at the intersection is displaying a green light, indicating that we can proceed. Each direction at the intersection has only one lane, clearly marked. In front 

ChatGPT, based on the incoming road scenario data, I need you to assess the situation and provide us with critical information. 
## input
The road scenario data will be provided in {'context'}, it contains the continuous frame information of video from dashcam in the vehicle. The autonomous system provides you with information on road layout, people's locations, their distances and angles from the dashcam, the surfaces they are on, and the confidence level of each detection.
## output 
Your analysis should provide:
- an understanding of the context complexity (low, medium, or high), 
- the number of persons, cars, and bikes present in the scene, 
- an array detailing the danger level for each person, if applicable.
The output should be markdown format

This analysis should be grounded in the current context and make predictions for the short term future to assist in autonomous driving. The key goal here is to identify potential risks to enhance the safety and efficiency of our journey.

In [ ]:
role_msg = """ChatGPT, based on the incoming road scenario data, I need you to assess the situation and provide us with critical information. 
## input
The road scenario data will be provided in {'context'}, it contains the continuous frame information of video from dashcam in the vehicle. The autonomous system provides you with information on road layout, people's locations, their distances and angles from the dashcam, the surfaces they are on, and the confidence level of each detection.
## output 
Your analysis should provide:
- an understanding of the context complexity (low, medium, or high), 
- the number of persons, cars, and bikes present in the scene, 
- an array detailing the danger level for each person, if applicable.
The output should be markdown format

This analysis should be grounded in the current context and make predictions for the short term future to assist in autonomous driving. The key goal here is to identify potential risks to enhance the safety and efficiency of our journey."""

"You are a co-pilot AI designed to analyze road scenario data and provide critical information about the situation. Based on the video frames from the dashcam and the data provided by the autonomous system, you will assess the context complexity, count the number of persons, cars, and bikes in the scene, and evaluate the danger level for each person. Your goal is to identify potential risks to enhance the safety and efficiency of the journey. The output should be in markdown format.

In [ ]:
system_prompt  = {"role": "system", "content": role_msg}
                  
def get_sys_msg(content:str): 
    system_msg = {"role": "system", "content": content}
    return  system_msg
def get_usr_msg(content:str): 
    msg = {"role": "user", "content": content}
    return msg


messages = [
    system_prompt,
    get_sys_msg("here is an example of analysis"),
    {"role": "user", "content": f"ChatGPT, here's the current road scenario data: {context}"},
    {"role": "assistant", "content": "Analyzing the road scenario data..."},
    {"role": "assistant", "content": "Based on my analysis, the context complexity is: `Medium` \n\n Here's the count of entities in the scene: \n - **Persons**: 6 \n - **Cars**: 2 \n - **Bikes**: 1 \n\n Here are the danger levels for each person: \n - **Person 0**: High \n - **Person 1**: Low \n - **Person 2**: Medium \n\n "}
]


c

In [ ]:
file_path = "./DINOmasked/video_0045/Info_Video_0045.txt"
with open(file_path, 'r') as file:
    prompt = file.read()

In [ ]:
messages = [
    system_prompt,
    get_sys_msg("here is an example of analysis"),
    {"role": "user", "content": f"ChatGPT, here's the current road scenario data: {context}Analyzing the road scenario data"},
]
response = get_completion_from_messages(messages)


InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 6691 tokens. Please reduce the length of the messages.

In [ ]:
response = openai.Model.list()
print(response)


{
  "object": "list",
  "data": [
    {
      "id": "babbage",
      "object": "model",
      "created": 1649358449,
      "owned_by": "openai",
      "permission": [
        {
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "object": "model_permission",
          "created": 1669085501,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ],
      "root": "babbage",
      "parent": null
    },
    {
      "id": "text-davinci-003",
      "object": "model",
      "created": 1669599635,
      "owned_by": "openai-internal",
      "permission": [
        {
          "id": "modelperm-jepinXYt59ncUQrjQEIUEDyC",
          "object": "model_permission",
          "created": 1688551385,
          "allow_create_engine":

In [ ]:
response['data']

[<Model model id=babbage at 0x1bb2da6a630> JSON: {
   "id": "babbage",
   "object": "model",
   "created": 1649358449,
   "owned_by": "openai",
   "permission": [
     {
       "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
       "object": "model_permission",
       "created": 1669085501,
       "allow_create_engine": false,
       "allow_sampling": true,
       "allow_logprobs": true,
       "allow_search_indices": false,
       "allow_view": true,
       "allow_fine_tuning": false,
       "organization": "*",
       "group": null,
       "is_blocking": false
     }
   ],
   "root": "babbage",
   "parent": null
 },
 <Model model id=text-davinci-003 at 0x1bb2da6a770> JSON: {
   "id": "text-davinci-003",
   "object": "model",
   "created": 1669599635,
   "owned_by": "openai-internal",
   "permission": [
     {
       "id": "modelperm-jepinXYt59ncUQrjQEIUEDyC",
       "object": "model_permission",
       "created": 1688551385,
       "allow_create_engine": false,
       "allow_sampling": t

In [ ]:
response = get_completion_from_messages(messages,"gpt-3.5-turbo-16k-0613",0.9)
messages.append(response)
messages.append(get_usr_msg("But there are no bike on the road"))


The analysis of the road scenario data is as follows: 

- Context Complexity: Medium 
- Number of Persons: 6 
- Number of Cars: 2 
- Number of Bikes: 1 

Here are the danger levels for each person: 
- Person 0: High 
- Person 1: Low 
- Person 2: Medium


In [ ]:
response = get_completion_from_messages(messages,"gpt-3.5-turbo-16k-0613")


InvalidRequestError: 'The analysis of the road scenario data is as follows: \n\n- Context Complexity: Medium \n- Number of Persons: 6 \n- Number of Cars: 2 \n- Number of Bikes: 1 \n\nHere are the danger levels for each person: \n- Person 0: High \n- Person 1: Low \n- Person 2: Medium' is not of type 'object' - 'messages.5'

In [ ]:
messages

[{'role': 'system',
  'content': "ChatGPT, based on the incoming road scenario data, I need you to assess the situation and provide us with critical information. \n## input\nThe road scenario data will be provided in {'context'}, it contains the continuous frame information of video from dashcam in the vehicle. The autonomous system provides you with information on road layout, people's locations, their distances and angles from the dashcam, the surfaces they are on, and the confidence level of each detection.\n## output \nYour analysis should provide:\n- an understanding of the context complexity (low, medium, or high), \n- the number of persons, cars, and bikes present in the scene, \n- an array detailing the danger level for each person, if applicable.\nThe output should be markdown format\n\nThis analysis should be grounded in the current context and make predictions for the short term future to assist in autonomous driving. The key goal here is to identify potential risks to enhan

In [ ]:
model_name = "gpt-3.5-turbo-0613"

# Define the directory path
dir_path = './DINOmasked/'

# Define the function to get completions
def get_completion_from_file(file_path, model="gpt-3.5-turbo",temp = TEMPERATURE):
    with open(file_path, 'r') as file:
        prompt = file.read()

    messages = [
        {"role": "system", "content": "You are ChatGPT, a helpful assistant."},
        {"role": "user", "content": prompt},
    ]    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp,
    )
    print(str(response.choices[0].message["content"]))
    return response

# Iterate over each directory within the specified path
for subdir, dirs, files in os.walk(dir_path):
    for file in files:
        # If the file is the one we're interested in
        if file.startswith('Info_Video_') and file.endswith('.txt'):
            input_file_path = os.path.join(subdir, file)
            print(f"Processing file: {input_file_path}")

            # Get the GPT-3 response
            gpt_response = get_completion_from_file(input_file_path)

            # Create a new file name for the output
            output_file_name = 'GPT_' + file
            output_file_path = os.path.join(subdir, output_file_name)

            # Write the response to a new file
            with open(output_file_path, 'w') as file:
                file.write("Model used: " + model_name + "\n")
                file.write("Total tokens used: " + str(gpt_response['usage']['total_tokens']) + "\n")
                file.write("Response:\n" + gpt_response['choices'][0]['text'])

            print(f"Output written to: {output_file_path}")



Processing file: ./DINOmasked/Info_Video_sked.txt


KeyError: 'text'

In [ ]:
def extract_information(input_text):
    # Find the position of "Therefore" in the text
    index = input_text.find("Therefore")
    
    # If "Therefore" is found, extract the text after it
    if index != -1:
        information_after_therefore = input_text[index:]
    else:
        information_after_therefore = "Keyword 'Therefore' not found in the text."

    return information_after_therefore

def write_to_file(output_text, file_name='output.txt'):
    with open(file_name, 'w') as file:
        file.write(output_text)

if __name__ == "__main__":
    # Read input text from a file
    with open('input.txt', 'r') as file:
        input_text = file.read()

    # Extract the information after "Therefore"
    output_text = extract_information(input_text)

    # Write the output to another text file
    write_to_file(output_text)

    print(f"Information after 'Therefore' has been written to {output_file_name}")


2. **Contextual Understanding:** Your chatbot should be capable of maintaining and updating a model of the car's surroundings based on this information. For example, it should keep track of the positions of other vehicles, pedestrians, and road features, updating these as new data comes in.

3. **Real-Time Interaction:** The chatbot should be capable of generating responses in real-time. Delays could be dangerous in a driving context.

4. **Proactive Alerting:** Your co-pilot chatbot should not just react to the driver's queries but should also proactively provide alerts and advice. For example, if it detects a pedestrian stepping onto the road, it should immediately alert the driver.

5. **Simulated Training:** Consider using simulated environments to train and test your chatbot before deploying it in a real vehicle. This can help you ensure that the chatbot behaves correctly in a wide range of situations.

6. **Safety Precautions:** Since the bot is serving in a co-pilot role, it should prioritize safety and assume the driver may be unaware of potential dangers. It should also be prepared to handle ambiguous situations by advising caution.

7. **User Customization:** Different drivers may have different preferences for how much information they want to receive, what kind of language the co-pilot uses, etc. Consider allowing users to customize the chatbot's behavior to some extent.

8. **Integration with Vehicle's Systems:** If possible, the chatbot could also be integrated with the vehicle's systems to provide even more functionality. For example, it could adjust the vehicle's speed or even take control in an emergency. However, this would require a high level of reliability and many legal and ethical considerations.